In [41]:
import torch
import torch.nn.functional as F
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [42]:
%load_ext autoreload
%autoreload 2

import numpy as np

from pytorchcv.model_provider import get_model as ptcv_get_model # model

import sys, os

import easydict

import torch
import torch.nn as nn
import numpy as np

import torch.backends.cudnn as cudnn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim

import matplotlib.pyplot as plt
#%matplotlib inline 

# enable cuda devices
import os    
os.environ['KMP_DUPLICATE_LIB_OK']='True'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 0. 사용할 Dataset 설정

In [43]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


from meta_neural_network_architectures import VGGReLUNormNetwork, ResNet12
from utils.parser_utils import get_args
from data import MetaLearningSystemDataLoader
from experiment_builder import ExperimentBuilder

from few_shot_learning_system import MAMLFewShotClassifier
from utils import basic_utils

csv_file = "sample_loss_log_Arbiter(miniImageNet_1shot).csv"

datasets = "mini_imagenet"
# datasets = "tiered_imagenet"
# datasets = "CIFAR_FS"
# datasets = "CUB"

os.environ['DATASET_DIR'] = 'C:/Users/JM/PycharmProjects/MAML/datasets'
os.environ['TEST_DATASET'] = "tiered_imagenet" # https://mtl.yyliu.net/download/Lmzjm9tX.html
# os.environ['TEST_DATASET'] = "CIFAR_FS" # https://drive.google.com/file/d/1pTsCCMDj45kzFYgrnO67BWVbKs48Q3NI/view
# os.environ['TEST_DATASET'] = "CUB" # https://data.caltech.edu/records/65de6-vp158

In [44]:
args = easydict.EasyDict(
{
  "batch_size":4,
  "image_height":84,
  "image_width":84,
  "image_channels":3,
  "gpu_to_use":0,
  "num_dataprovider_workers":4,
  "max_models_to_save":5,
  "dataset_name":"mini_imagenet_full_size",
  "dataset_path":"mini_imagenet_full_size",
  "reset_stored_paths":False,
  "experiment_name":"../MAML+Arbiter_5way_1shot",
  "train_seed": 0, "val_seed": 0,
  "indexes_of_folders_indicating_class": [-3, -2],
  "sets_are_pre_split": True,
  "train_val_test_split": [0.64, 0.16, 0.20],
  "evaluate_on_test_set_only": False,

  "total_epochs": 150,
  "total_iter_per_epoch":500, "continue_from_epoch": -2,
  "num_evaluation_tasks":600,
  "multi_step_loss_num_epochs": 15,
  "minimum_per_task_contribution": 0.01,
  "learnable_per_layer_per_step_inner_loop_learning_rate": False,
  "enable_inner_loop_optimizable_bn_params": False,
  "evalute_on_test_set_only": False,

  "max_pooling": True,
  "per_step_bn_statistics": False,
  "learnable_batch_norm_momentum": False,
  "load_into_memory": False,
  "init_inner_loop_learning_rate": 0.01,
  "init_inner_loop_weight_decay": 0.0005,
  "learnable_bn_gamma": True,
  "learnable_bn_beta": True,

  "dropout_rate_value":0.0,
  "min_learning_rate":0.0001,
  "meta_learning_rate":0.0001,   "total_epochs_before_pause": 151,
  "first_order_to_second_order_epoch":-1,
  "weight_decay": 0.0,

  "norm_layer":"batch_norm",
  "cnn_num_filters":128,
  "num_stages":4,
  "conv_padding": True,
  "number_of_training_steps_per_iter":5,
  "number_of_evaluation_steps_per_iter":5,
  "cnn_blocks_per_stage":1,
  "num_classes_per_set":5,
  "num_samples_per_class":1,
  "num_target_samples": 15,
  "samples_per_iter" : 1,

  "second_order": True,
  "use_multi_step_loss_optimization":False,
  "backbone": "4-CONV",
  "arbiter": True,
  "use_bias": True,
  "smoothing": False,
  "knowledge_distillation": False,
  "momentum": "SGD"
}
)

device = torch.cuda.current_device()
args.im_shape = (2, 3, args.image_height, args.image_width)

args.use_cuda = torch.cuda.is_available()
args.seed = 104
args.reverse_channels=False
args.labels_as_int=False
args.reset_stored_filepaths=False
args.num_of_gpus=1

args.continue_from_epoch='latest'

In [45]:
# 모델을 구성한다
model = MAMLFewShotClassifier(args=args, device=device,
                              im_shape=(2, 3,
                                        args.image_height, args.image_width))

data = MetaLearningSystemDataLoader

maml_system = ExperimentBuilder(model=model, data=data, args=args, device=device)

Using max pooling
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
Inner Loop parameters
Outer Loop parameters
log_dir ===  C:\Users\JM\PycharmProjects\MAML\MAML+Arbiter_5way_1shot
attempting to find existing checkpoint
dataset_splits ==  dict_keys(['test', 'train', 'val'])
data {'test': 206209, 'train': 448695, 'val': 124261}
train_seed 985773, val_seed: 985773, at start time
75000 75000


In [46]:
maml_system.state

{'best_val_acc': 0.5149999976158142,
 'best_val_iter': 63500,
 'current_iter': 75000,
 'best_epoch': 127,
 'train_loss_mean': 0.6405333166122437,
 'train_loss_std': 0.12306425183415713,
 'train_accuracy_mean': 0.7643866658210754,
 'train_accuracy_std': 0.05608368177868576,
 'train_loss_importance_vector_0_mean': 0.006000000052154064,
 'train_loss_importance_vector_0_std': 0.0,
 'train_loss_importance_vector_1_mean': 0.006000000052154064,
 'train_loss_importance_vector_1_std': 0.0,
 'train_loss_importance_vector_2_mean': 0.006000000052154064,
 'train_loss_importance_vector_2_std': 0.0,
 'train_loss_importance_vector_3_mean': 0.006000000052154064,
 'train_loss_importance_vector_3_std': 0.0,
 'train_loss_importance_vector_4_mean': 0.9760000109672546,
 'train_loss_importance_vector_4_std': 0.0,
 'train_learning_rate_mean': 0.00010000000000000003,
 'train_learning_rate_std': 2.710505431213761e-20,
 'val_loss_mean': 1.256844596862793,
 'val_loss_std': 0.13286672963334395,
 'val_accuracy_mean

# 1. 학습된 모델을 불러온다

In [47]:
model_save_dir = maml_system.saved_models_filepath
model_name = "train_model"
model_idx = maml_system.state['best_epoch']

state = maml_system.model.load_model(model_save_dir=model_save_dir,
                                     model_name=model_name,
                                     model_idx=model_idx+1)

state_dict_loaded = state['network']

maml_system.model.load_state_dict(state_dict=state_dict_loaded)

# # 잘 불러왔는지 확인하는 코드
# print("state_dict_loaded == ",state_dict_loaded)
# print("="*10)
# for key, value in maml_system.model.named_parameters():
#     print(key)
#     print(value)
maml_system.model

MAMLFewShotClassifier(
  (classifier): VGGReLUNormNetwork(
    (layer_dict): ModuleDict(
      (conv0): MetaConvNormLayerReLU(
        (layer_dict): ModuleDict()
        (conv): MetaConv2dLayer()
        (norm_layer): MetaBatchNormLayer(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv1): MetaConvNormLayerReLU(
        (layer_dict): ModuleDict()
        (conv): MetaConv2dLayer()
        (norm_layer): MetaBatchNormLayer(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv2): MetaConvNormLayerReLU(
        (layer_dict): ModuleDict()
        (conv): MetaConv2dLayer()
        (norm_layer): MetaBatchNormLayer(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv3): MetaConvNormLayerReLU(
        (layer_dict): ModuleDict()
        (conv): MetaConv2dLayer()
        (norm_layer): MetaBatchNormLayer(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (line

# 2. Samlpe 별 loss 구현

In [48]:
train_data = maml_system.data.get_train_batches(total_batches=int(600/2), augment_images=False)

task_idx = 0

for sample_idx, train_sample in enumerate(train_data):
    
    x_support_set, x_target_set, y_support_set, y_target_set, seed = train_sample
    
    x_support_set = torch.Tensor(x_support_set).float().to(device=maml_system.model.device)
    x_target_set = torch.Tensor(x_target_set).float().to(device=maml_system.model.device)
    y_support_set = torch.Tensor(y_support_set).long().to(device=maml_system.model.device)
    y_target_set = torch.Tensor(y_target_set).long().to(device=maml_system.model.device)
    
    for task_id, (x_support_set_task, y_support_set_task, x_target_set_task, y_target_set_task) in enumerate(zip(x_support_set,
                              y_support_set,
                              x_target_set,
                              y_target_set)):
        
        names_weights_copy = maml_system.model.get_inner_loop_parameter_dict(maml_system.model.classifier.named_parameters())
        
        
        num_devices = torch.cuda.device_count() if torch.cuda.is_available() else 1

        names_weights_copy = {
            name.replace('module.', ''): value.unsqueeze(0).repeat(
                [num_devices] + [1 for i in range(len(value.shape))]) for
            name, value in names_weights_copy.items()}
        
        n, s, c, h, w = x_target_set_task.shape

        x_support_set_task = x_support_set_task.view(-1, c, h, w)
        y_support_set_task = y_support_set_task.view(-1)
        x_target_set_task = x_target_set_task.view(-1, c, h, w)
        y_target_set_task = y_target_set_task.view(-1)
        
        task_idx = task_idx + 1
        print(f"Calculating the loss of {task_idx}")
        
        num_steps=5
        for num_step in range(num_steps):            
            support_loss, support_preds, support_loss_seperate, fetaure_map = maml_system.model.net_forward(
                    x=x_support_set_task,
                    y=y_support_set_task,
                    weights=names_weights_copy,
                    backup_running_statistics=num_step == 0,
                    training=True,
                    num_step=num_step,
                    training_phase='test',
                    epoch=0
                )
            
            
            # 데이터프레임 생성
            df = pd.DataFrame({
                'Task_Idx' : [task_idx] * len(support_loss_seperate),
                'Step': [num_step] * len(support_loss_seperate),
                'Sample_Index': list(range(len(support_loss_seperate))),
                'Loss': support_loss_seperate.detach().cpu().numpy()
            })

            # 파일이 존재하지 않으면 새로운 파일 생성 (mode='w')
            if not os.path.exists(csv_file):
                df.to_csv(csv_file, mode='w', index=False, encoding='utf-8')
            else:
                # 파일이 존재하면 append (mode='a')
                df.to_csv(csv_file, mode='a', header=False, index=False, encoding='utf-8')
        
            generated_alpha_params = {}

            if maml_system.model.args.arbiter:
                support_loss_grad = torch.autograd.grad(support_loss, names_weights_copy.values(),
                                                        retain_graph=True)

                names_grads_copy = dict(zip(names_weights_copy.keys(), support_loss_grad))

                per_step_task_embedding = []

                for key, weight in names_weights_copy.items():
                    weight_norm = torch.norm(weight, p=2)
                    per_step_task_embedding.append(weight_norm)

                for key, grad in names_grads_copy.items():
                    gradient_l2norm = torch.norm(grad, p=2)
                    per_step_task_embedding.append(gradient_l2norm)

                per_step_task_embedding = torch.stack(per_step_task_embedding)

                per_step_task_embedding = (per_step_task_embedding - per_step_task_embedding.mean()) / (
                            per_step_task_embedding.std() + 1e-12)

                generated_gradient_rate = maml_system.model.arbiter(per_step_task_embedding)

                g = 0
                for key in names_weights_copy.keys():
                    generated_alpha_params[key] = generated_gradient_rate[g]
                    g += 1

            names_weights_copy, names_grads_copy = maml_system.model.apply_inner_loop_update(
                loss=support_loss,
                support_loss_seperate=support_loss_seperate,
                names_weights_copy=names_weights_copy,
                alpha=generated_alpha_params,
                use_second_order=args.second_order,
                current_step_idx=num_step,
                current_iter=maml_system.state['current_iter'],
                training_phase='test')
                      
#             if num_step==4:
#                 target_loss, target_preds, _, _ = self.net_forward(x=x_target_set_task,
#                                                                  y=y_target_set_task, weights=names_weights_copy,
#                                                                  backup_running_statistics=False, training=True,
#                                                                  num_step=num_step, training_phase=training_phase,
#                                                                  epoch=epoch)

        ##########Inner loop 종료 ############

print("Loss_per_sample_Arbiter End")

Calculating the loss of 1
Calculating the loss of 2
Calculating the loss of 3
Calculating the loss of 4
Calculating the loss of 5
Calculating the loss of 6
Calculating the loss of 7
Calculating the loss of 8
Calculating the loss of 9
Calculating the loss of 10
Calculating the loss of 11
Calculating the loss of 12
Calculating the loss of 13
Calculating the loss of 14
Calculating the loss of 15
Calculating the loss of 16
Calculating the loss of 17
Calculating the loss of 18
Calculating the loss of 19
Calculating the loss of 20
Calculating the loss of 21
Calculating the loss of 22
Calculating the loss of 23
Calculating the loss of 24
Calculating the loss of 25
Calculating the loss of 26
Calculating the loss of 27
Calculating the loss of 28
Calculating the loss of 29
Calculating the loss of 30
Calculating the loss of 31
Calculating the loss of 32
Calculating the loss of 33
Calculating the loss of 34
Calculating the loss of 35
Calculating the loss of 36
Calculating the loss of 37
Calculatin

Calculating the loss of 298
Calculating the loss of 299
Calculating the loss of 300
Calculating the loss of 301
Calculating the loss of 302
Calculating the loss of 303
Calculating the loss of 304
Calculating the loss of 305
Calculating the loss of 306
Calculating the loss of 307
Calculating the loss of 308
Calculating the loss of 309
Calculating the loss of 310
Calculating the loss of 311
Calculating the loss of 312
Calculating the loss of 313
Calculating the loss of 314
Calculating the loss of 315
Calculating the loss of 316
Calculating the loss of 317
Calculating the loss of 318
Calculating the loss of 319
Calculating the loss of 320
Calculating the loss of 321
Calculating the loss of 322
Calculating the loss of 323
Calculating the loss of 324
Calculating the loss of 325
Calculating the loss of 326
Calculating the loss of 327
Calculating the loss of 328
Calculating the loss of 329
Calculating the loss of 330
Calculating the loss of 331
Calculating the loss of 332
Calculating the loss

Calculating the loss of 592
Calculating the loss of 593
Calculating the loss of 594
Calculating the loss of 595
Calculating the loss of 596
Calculating the loss of 597
Calculating the loss of 598
Calculating the loss of 599
Calculating the loss of 600
Calculating the loss of 601
Calculating the loss of 602
Calculating the loss of 603
Calculating the loss of 604
Calculating the loss of 605
Calculating the loss of 606
Calculating the loss of 607
Calculating the loss of 608
Calculating the loss of 609
Calculating the loss of 610
Calculating the loss of 611
Calculating the loss of 612
Calculating the loss of 613
Calculating the loss of 614
Calculating the loss of 615
Calculating the loss of 616
Calculating the loss of 617
Calculating the loss of 618
Calculating the loss of 619
Calculating the loss of 620
Calculating the loss of 621
Calculating the loss of 622
Calculating the loss of 623
Calculating the loss of 624
Calculating the loss of 625
Calculating the loss of 626
Calculating the loss

Calculating the loss of 885
Calculating the loss of 886
Calculating the loss of 887
Calculating the loss of 888
Calculating the loss of 889
Calculating the loss of 890
Calculating the loss of 891
Calculating the loss of 892
Calculating the loss of 893
Calculating the loss of 894
Calculating the loss of 895
Calculating the loss of 896
Calculating the loss of 897
Calculating the loss of 898
Calculating the loss of 899
Calculating the loss of 900
Calculating the loss of 901
Calculating the loss of 902
Calculating the loss of 903
Calculating the loss of 904
Calculating the loss of 905
Calculating the loss of 906
Calculating the loss of 907
Calculating the loss of 908
Calculating the loss of 909
Calculating the loss of 910
Calculating the loss of 911
Calculating the loss of 912
Calculating the loss of 913
Calculating the loss of 914
Calculating the loss of 915
Calculating the loss of 916
Calculating the loss of 917
Calculating the loss of 918
Calculating the loss of 919
Calculating the loss

Calculating the loss of 1174
Calculating the loss of 1175
Calculating the loss of 1176
Calculating the loss of 1177
Calculating the loss of 1178
Calculating the loss of 1179
Calculating the loss of 1180
Calculating the loss of 1181
Calculating the loss of 1182
Calculating the loss of 1183
Calculating the loss of 1184
Calculating the loss of 1185
Calculating the loss of 1186
Calculating the loss of 1187
Calculating the loss of 1188
Calculating the loss of 1189
Calculating the loss of 1190
Calculating the loss of 1191
Calculating the loss of 1192
Calculating the loss of 1193
Calculating the loss of 1194
Calculating the loss of 1195
Calculating the loss of 1196
Calculating the loss of 1197
Calculating the loss of 1198
Calculating the loss of 1199
Calculating the loss of 1200
Loss_per_sample_Arbiter End
